In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

data = pd.read_csv('iris.csv', header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])

class_mapping = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
data['class'] = data['class'].map(class_mapping)

X = data.iloc[:, :-1]
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear', C=1.0)

svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report_str)


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [13]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import ParameterEstimator, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD

model = BayesianModel([('ExamLevel', 'Marks'), ('IQLevel', 'Marks'),('IQLevel', 'AptiScore'), ('Marks', 'Admission')])

cpt_exam_level = TabularCPD(variable='ExamLevel', variable_card=2, values=[[0.7], [0.3]])

cpt_iq_level = TabularCPD(variable='IQLevel', variable_card=2, values=[[0.8], [0.2]])

cpt_marks = TabularCPD(variable='Marks', variable_card=2,
                       values=[[0.6, 0.9, 0.5, 0.8],[0.4, 0.1, 0.5, 0.2]],
                       evidence=['IQLevel', 'ExamLevel'],
                       evidence_card=[2, 2])

cpt_apti_score = TabularCPD(variable='AptiScore', variable_card=2,
                            values=[[0.75, 0.4],[0.25, 0.6]],
                            evidence=['IQLevel'],
                            evidence_card=[2])

cpt_admission = TabularCPD(variable='Admission', variable_card=2,
                           values=[[0.6, 0.9],[0.4, 0.1]],
                           evidence=['Marks'],
                           evidence_card=[2])

model.add_cpds(cpt_exam_level, cpt_iq_level, cpt_marks,cpt_apti_score, cpt_admission)

print("Model is valid:", model.check_model())

inference = VariableElimination(model)
result = inference.query(variables=['Marks'], evidence={'ExamLevel': 1})
print(result)


Model is valid: True
+----------+--------------+
| Marks    |   phi(Marks) |
+==========+==============+
| Marks(0) |       0.8800 |
+----------+--------------+
| Marks(1) |       0.1200 |
+----------+--------------+
